# Load Library & Config

## Library

In [1]:
#!/usr/bin/env python
# coding=utf-8
# Copyright 2020 The HuggingFace Inc. team. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""
Fine-tuning the library models for causal language modeling (GPT, GPT-2, CTRL, ...) on a text file or a dataset.

Here is the full list of checkpoints on the hub that can be fine-tuned by this script:
https://huggingface.co/models?filter=text-generation
"""
# You can also adapt this script on your own causal language modeling task. Pointers for this are left as comments.

import logging
import math
import os
import sys
from dataclasses import dataclass, field
from itertools import chain
from typing import Optional

import datasets
import evaluate
import torch
from datasets import load_dataset

import transformers
from transformers import (
    CONFIG_MAPPING,
    MODEL_FOR_CAUSAL_LM_MAPPING,
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    default_data_collator,
    is_torch_tpu_available,
    set_seed,
)
from transformers.testing_utils import CaptureLogger
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version, send_example_telemetry
from transformers.utils.versions import require_version
from transformers import EarlyStoppingCallback

## Congfig

### ModelArguments, DataArguments

In [2]:
# Will error if the minimal version of Transformers is not installed. Remove at your own risks.
# check_min_version("4.30.0.dev0")

require_version("datasets>=1.8.0", "To fix: pip install -r examples/pytorch/language-modeling/requirements.txt")

logger = logging.getLogger(__name__)


MODEL_CONFIG_CLASSES = list(MODEL_FOR_CAUSAL_LM_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)


# 없는 arguments는 여기에 추가하면 될듯??
@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune, or train from scratch.
    """

    model_name_or_path: Optional[str] = field(
        default=None,
        metadata={
            "help": (
                "The model checkpoint for weights initialization.Don't set if you want to train a model from scratch."
            )
        },
    )
    model_type: Optional[str] = field(
        default=None,
        metadata={"help": "If training from scratch, pass a model type from the list: " + ", ".join(MODEL_TYPES)},
    )
    config_overrides: Optional[str] = field(
        default=None,
        metadata={
            "help": (
                "Override some existing default config settings when a model is trained from scratch. Example: "
                "n_embd=10,resid_pdrop=0.2,scale_attn_weights=false,summary_type=cls_index"
            )
        },
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    cache_dir: Optional[str] = field(
        default=None,
        metadata={"help": "Where do you want to store the pretrained models downloaded from huggingface.co"},
    )
    use_fast_tokenizer: bool = field(
        default=True,
        metadata={"help": "Whether to use one of the fast tokenizer (backed by the tokenizers library) or not."},
    )
    model_revision: str = field(
        default="main",
        metadata={"help": "The specific model version to use (can be a branch name, tag name or commit id)."},
    )
    use_auth_token: bool = field(
        default=False,
        metadata={
            "help": (
                "Will use the token generated when running `huggingface-cli login` (necessary to use this script "
                "with private models)."
            )
        },
    )
    torch_dtype: Optional[str] = field(
        default=None,
        metadata={
            "help": (
                "Override the default `torch.dtype` and load the model under this dtype. If `auto` is passed, the "
                "dtype will be automatically derived from the model's weights."
            ),
            "choices": ["auto", "bfloat16", "float16", "float32"],
        },
    )
    low_cpu_mem_usage: bool = field(
        default=False,
        metadata={
            "help": (
                "It is an option to create the model as an empty shell, then only materialize its parameters when the pretrained weights are loaded."
                "set True will benefit LLM loading time and RAM consumption."
            )
        },
    )

    def __post_init__(self):
        if self.config_overrides is not None and (self.config_name is not None or self.model_name_or_path is not None):
            raise ValueError(
                "--config_overrides can't be used in combination with --config_name or --model_name_or_path"
            )


@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """

    dataset_name: Optional[str] = field(
        default=None, metadata={"help": "The name of the dataset to use (via the datasets library)."}
    )
    dataset_config_name: Optional[str] = field(
        default=None, metadata={"help": "The configuration name of the dataset to use (via the datasets library)."}
    )
    train_file: Optional[str] = field(default=None, metadata={"help": "The input training data file (a text file)."})
    validation_file: Optional[str] = field(
        default=None,
        metadata={"help": "An optional input evaluation data file to evaluate the perplexity on (a text file)."},
    )
    max_train_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": (
                "For debugging purposes or quicker training, truncate the number of training examples to this "
                "value if set."
            )
        },
    )
    max_eval_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": (
                "For debugging purposes or quicker training, truncate the number of evaluation examples to this "
                "value if set."
            )
        },
    )
    streaming: bool = field(default=False, metadata={"help": "Enable streaming mode"})
    block_size: Optional[int] = field(
        default=None,
        metadata={
            "help": (
                "Optional input sequence length after tokenization. "
                "The training dataset will be truncated in block of this size for training. "
                "Default to the model max input length for single sentence inputs (take into account special tokens)."
            )
        },
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached training and evaluation sets"}
    )
    validation_split_percentage: Optional[int] = field(
        default=5,
        metadata={
            "help": "The percentage of the train set used as validation set in case there's no validation split"
        },
    )
    preprocessing_num_workers: Optional[int] = field(
        default=None,
        metadata={"help": "The number of processes to use for the preprocessing."},
    )
    keep_linebreaks: bool = field(
        default=True, metadata={"help": "Whether to keep line breaks when using TXT files or not."}
    )

    def __post_init__(self):
        if self.streaming:
            require_version("datasets>=2.0.0", "The streaming feature requires `datasets>=2.0.0`")

        if self.dataset_name is None and self.train_file is None and self.validation_file is None:
            raise ValueError("Need either a dataset name or a training/validation file.")
        else:
            if self.train_file is not None:
                extension = self.train_file.split(".")[-1]
                assert extension in ["csv", "json", "txt"], "`train_file` should be a csv, a json or a txt file."
            if self.validation_file is not None:
                extension = self.validation_file.split(".")[-1]
                assert extension in ["csv", "json", "txt"], "`validation_file` should be a csv, a json or a txt file."




### Load and Parse Config 

In [3]:
# See all possible arguments in src/transformers/training_args.py
# or by passing the --help flag to this script.
# We now keep distinct sets of args, for a cleaner separation of concerns.

# 모든 데이터를 json file에 박아 놓으면 위에 class setting 해놓은거에 따라서 잘 parsing 됨
# 테스트: 1) 위에 클래스에 arguement 추가, 2) Training arguemnt는 추가가 안되는데 어떡하지?
parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TrainingArguments))

model_args, data_args, training_args = parser.parse_json_file(json_file="config_sample.json")

send_example_telemetry("run_clm", model_args, data_args)

### logging 

In [4]:
# Setup logging
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
)

if training_args.should_log:
    # The default of training_args.log_level is passive, so we set log level at info here to have that default.
    transformers.utils.logging.set_verbosity_info()

log_level = training_args.get_process_log_level()
logger.setLevel(log_level)
datasets.utils.logging.set_verbosity(log_level)
transformers.utils.logging.set_verbosity(log_level)
transformers.utils.logging.enable_default_handler()
transformers.utils.logging.enable_explicit_format()

# Log on each process the small summary:
logger.warning(
    f"Process rank: {training_args.local_rank}, device: {training_args.device}, n_gpu: {training_args.n_gpu}"
    + f"distributed training: {bool(training_args.local_rank != -1)}, 16-bits training: {training_args.fp16}"
)
logger.info(f"Training/evaluation parameters {training_args}")



06/09/2023 17:25:07 - WARNING - __main__ - Process rank: -1, device: mps, n_gpu: 1distributed training: False, 16-bits training: False
06/09/2023 17:25:07 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.99,
adam_epsilon=1e-06,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=1000,
evaluation_strategy=steps,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_step

### checkpoint 

In [5]:
# output_dir 기준으로 기존에 training 한 이력이 있는지 확인
# get_last_checkpoint를 통해 마지막 checkpoint 확인 (resume_from_checkpoint로 설정해도 됨)
# 만약 overwrite 관련해서 문제 있으면 overwrite_output_dir True 시키라고 함 문제없으면 그냥 넘어감
# Detecting last checkpoint.
last_checkpoint = None
if os.path.isdir(training_args.output_dir) and training_args.do_train and not training_args.overwrite_output_dir:
    last_checkpoint = get_last_checkpoint(training_args.output_dir)
    if last_checkpoint is None and len(os.listdir(training_args.output_dir)) > 0:
        raise ValueError(
            f"Output directory ({training_args.output_dir}) already exists and is not empty. "
            "Use --overwrite_output_dir to overcome."
        )
    elif last_checkpoint is not None and training_args.resume_from_checkpoint is None:
        logger.info(
            f"Checkpoint detected, resuming training at {last_checkpoint}. To avoid this behavior, change "
            "the `--output_dir` or add `--overwrite_output_dir` to train from scratch."
        )

### seed

In [6]:
# seed 설정
# jupyter notebook 에서는 모든 셀에 이걸 넣어줘야함
# Set seed before initializing model.
set_seed(training_args.seed)

# Load Data, Model, Tokeniser

## Data

In [7]:
set_seed(training_args.seed)

# datasets 불러오는 과정 (datahub or offline)
# datahub: dataset_name, dataset_config_name, cache_dir (validation 없으면 validation_split_percentage 따라 나눔)
# offline: train_file, validation_file (확장자 .txt, .json, .csv 구별)

# Get the datasets: you can either provide your own CSV/JSON/TXT training and evaluation files (see below)
# or just provide the name of one of the public datasets available on the hub at https://huggingface.co/datasets/
# (the dataset will be downloaded automatically from the datasets Hub).
#
# For CSV/JSON files, this script will use the column called 'text' or the first column if no column called
# 'text' is found. You can easily tweak this behavior (see below).
#
# In distributed training, the load_dataset function guarantee that only one local process can concurrently
# download the dataset.
if data_args.dataset_name is not None:
    # Downloading and loading a dataset from the hub.
    raw_datasets = load_dataset(
        data_args.dataset_name,
        data_args.dataset_config_name,
        cache_dir=model_args.cache_dir,
        use_auth_token=True if model_args.use_auth_token else None,
        streaming=data_args.streaming,
    )
    if "validation" not in raw_datasets.keys():
        raw_datasets["validation"] = load_dataset(
            data_args.dataset_name,
            data_args.dataset_config_name,
            split=f"train[:{data_args.validation_split_percentage}%]",
            cache_dir=model_args.cache_dir,
            use_auth_token=True if model_args.use_auth_token else None,
            streaming=data_args.streaming,
        )
        raw_datasets["train"] = load_dataset(
            data_args.dataset_name,
            data_args.dataset_config_name,
            split=f"train[{data_args.validation_split_percentage}%:]",
            cache_dir=model_args.cache_dir,
            use_auth_token=True if model_args.use_auth_token else None,
            streaming=data_args.streaming,
        )
else:
    data_files = {}
    dataset_args = {}
    if data_args.train_file is not None:
        data_files["train"] = data_args.train_file
    if data_args.validation_file is not None:
        data_files["validation"] = data_args.validation_file
    extension = (
        data_args.train_file.split(".")[-1]
        if data_args.train_file is not None
        else data_args.validation_file.split(".")[-1]
    )
    if extension == "txt":
        extension = "text"
        dataset_args["keep_linebreaks"] = data_args.keep_linebreaks
    raw_datasets = load_dataset(
        extension,
        data_files=data_files,
        cache_dir=model_args.cache_dir,
        use_auth_token=True if model_args.use_auth_token else None,
        **dataset_args,
    )
    # If no validation data is there, validation_split_percentage will be used to divide the dataset.
    if "validation" not in raw_datasets.keys():
        raw_datasets["validation"] = load_dataset(
            extension,
            data_files=data_files,
            split=f"train[:{data_args.validation_split_percentage}%]",
            cache_dir=model_args.cache_dir,
            use_auth_token=True if model_args.use_auth_token else None,
            **dataset_args,
        )
        raw_datasets["train"] = load_dataset(
            extension,
            data_files=data_files,
            split=f"train[{data_args.validation_split_percentage}%:]",
            cache_dir=model_args.cache_dir,
            use_auth_token=True if model_args.use_auth_token else None,
            **dataset_args,
        )

# See more about loading any type of standard or custom dataset (from files, python dict, pandas DataFrame, etc) at
# https://huggingface.co/docs/datasets/loading_datasets.html.

06/09/2023 17:25:08 - INFO - datasets.info - Loading Dataset Infos from /Users/joon/.cache/huggingface/modules/datasets_modules/datasets/machelreid--m2d2/eb235f33a5de3163c10549b7f63c906910539c8a8c0ec5ade1285ccbf5067d00
06/09/2023 17:25:08 - INFO - datasets.builder - Overwrite dataset info from restored data version if exists.
06/09/2023 17:25:08 - INFO - datasets.info - Loading Dataset info from /Users/joon/.cache/huggingface/datasets/machelreid___m2d2/Religion_and_belief_systems/0.0.0/eb235f33a5de3163c10549b7f63c906910539c8a8c0ec5ade1285ccbf5067d00
06/09/2023 17:25:08 - WARNING - datasets.builder - Found cached dataset m2d2 (/Users/joon/.cache/huggingface/datasets/machelreid___m2d2/Religion_and_belief_systems/0.0.0/eb235f33a5de3163c10549b7f63c906910539c8a8c0ec5ade1285ccbf5067d00)
06/09/2023 17:25:08 - INFO - datasets.info - Loading Dataset info from /Users/joon/.cache/huggingface/datasets/machelreid___m2d2/Religion_and_belief_systems/0.0.0/eb235f33a5de3163c10549b7f63c906910539c8a8c0ec

  0%|          | 0/3 [00:00<?, ?it/s]

## model config

In [8]:
set_seed(training_args.seed)

# 모델, tokeniser loading
# 모델:
#   1) config 불러오기: config_name, model_name_or_path, model_type 따라 달라짐
#   2) model 불러오기
#   model_revision을 사용하면 branch 같이 작동하는것 같은데 한번 알아봐야겠음 TODO
# tokeniser: tokeniser 불러오기: tokenizer_name, model_name_or_path 따라 달라짐

# Load pretrained model and tokenizer
#
# Distributed training:
# The .from_pretrained methods guarantee that only one local process can concurrently
# download model & vocab.

config_kwargs = {
    "cache_dir": model_args.cache_dir,
    "revision": model_args.model_revision,
    "use_auth_token": True if model_args.use_auth_token else None,
}
if model_args.config_name:
    config = AutoConfig.from_pretrained(model_args.config_name, **config_kwargs)
elif model_args.model_name_or_path:
    config = AutoConfig.from_pretrained(model_args.model_name_or_path, **config_kwargs)
else:
    config = CONFIG_MAPPING[model_args.model_type]()
    logger.warning("You are instantiating a new config instance from scratch.")
    if model_args.config_overrides is not None:
        logger.info(f"Overriding config: {model_args.config_overrides}")
        config.update_from_string(model_args.config_overrides)
        logger.info(f"New config: {config}")

[INFO|configuration_utils.py:668] 2023-06-09 17:25:08,932 >> loading configuration file config.json from cache at /Users/joon/.cache/huggingface/hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/config.json
[INFO|configuration_utils.py:720] 2023-06-09 17:25:08,934 >> Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "t

## tokeniser

In [9]:
set_seed(training_args.seed)

tokenizer_kwargs = {
    "cache_dir": model_args.cache_dir,
    "use_fast": model_args.use_fast_tokenizer,
    "revision": model_args.model_revision,
    "use_auth_token": True if model_args.use_auth_token else None,
}
if model_args.tokenizer_name:
    tokenizer = AutoTokenizer.from_pretrained(model_args.tokenizer_name, **tokenizer_kwargs)
elif model_args.model_name_or_path:
    tokenizer = AutoTokenizer.from_pretrained(model_args.model_name_or_path, **tokenizer_kwargs)
else:
    raise ValueError(
        "You are instantiating a new tokenizer from scratch. This is not supported by this script."
        "You can do it from another script, save it, and load it from here, using --tokenizer_name."
    )

[INFO|tokenization_auto.py:502] 2023-06-09 17:25:09,074 >> Could not locate the tokenizer configuration file, will try to use the model config instead.
[INFO|configuration_utils.py:668] 2023-06-09 17:25:09,205 >> loading configuration file config.json from cache at /Users/joon/.cache/huggingface/hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/config.json
[INFO|configuration_utils.py:720] 2023-06-09 17:25:09,207 >> Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summ

## model

In [10]:
set_seed(training_args.seed)

if model_args.model_name_or_path:
    torch_dtype = (
        model_args.torch_dtype
        if model_args.torch_dtype in ["auto", None]
        else getattr(torch, model_args.torch_dtype)
    )
    model = AutoModelForCausalLM.from_pretrained(
        model_args.model_name_or_path,
        from_tf=bool(".ckpt" in model_args.model_name_or_path),
        config=config,
        cache_dir=model_args.cache_dir,
        revision=model_args.model_revision,
        use_auth_token=True if model_args.use_auth_token else None,
        torch_dtype=torch_dtype,
        low_cpu_mem_usage=model_args.low_cpu_mem_usage,
    )
else:
    model = AutoModelForCausalLM.from_config(config)
    n_params = sum({p.data_ptr(): p.numel() for p in model.parameters()}.values())
    logger.info(f"Training new model from scratch - Total size={n_params/2**20:.2f}M params")


[INFO|modeling_utils.py:2534] 2023-06-09 17:25:09,582 >> loading weights file pytorch_model.bin from cache at /Users/joon/.cache/huggingface/hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/pytorch_model.bin
[INFO|configuration_utils.py:575] 2023-06-09 17:25:09,760 >> Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.28.1"
}

[INFO|modeling_utils.py:3190] 2023-06-09 17:25:12,336 >> All model checkpoint weights were used when initializing GPT2LMHeadModel.

[INFO|modeling_utils.py:3198] 2023-06-09 17:25:12,337 >> All the weights of GPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GPT2LMHeadModel for predictions without further training.
[INFO|configuration_utils.py:537] 2023-06-09 17:25:12,472 >> loading configuration file generation_config.json from cache at 

# Tokenisation

In [11]:
# Tokenisation
#   1) embedding 사이즈 수정: (정확히는 모르겠는데) 모델과 토크나이저의 token 수가 다른 경우 index error 가 발생할 수 있는걸 방지
#   2) text_column_name: text_column_name 설정 (default: text, 없으면 [0]번째 column name)
#   3) tokenized_datasets 구성: 어떤 text_column_name 기준으로 어떻게 tokenize 할지 설정
#       streaming 여부, preprocessing_num_workers, overwrite_cache 설정 가능
#   3) group_texts: block_size에 따라서 텍스트 그룹화 시킴 (null일 경우 tokenizer.model_max_length) TODO
#   4) tokenisation: do_train, do_eval 에 따른 dataset tokenising (do_train is False 인 경우 train set은 안하고 건너뜀 (streaming preprocessing_num_workers 도 설정 가능))

## embedding size

In [12]:
set_seed(training_args.seed)

# We resize the embeddings only when necessary to avoid index errors. If you are creating a model from scratch
# on a small vocab and want a smaller embedding size, remove this test.
embedding_size = model.get_input_embeddings().weight.shape[0]
if len(tokenizer) > embedding_size:
    model.resize_token_embeddings(len(tokenizer))


## text_column_name

In [13]:
set_seed(training_args.seed)

# Preprocessing the datasets.
# First we tokenize all the texts.
if training_args.do_train:
    column_names = list(raw_datasets["train"].features)
else:
    column_names = list(raw_datasets["validation"].features)
text_column_name = "text" if "text" in column_names else column_names[0]


## tokenized_datasets

In [14]:
set_seed(training_args.seed)

# since this will be pickled to avoid _LazyModule error in Hasher force logger loading before tokenize_function
tok_logger = transformers.utils.logging.get_logger("transformers.tokenization_utils_base")

def tokenize_function(examples):
    with CaptureLogger(tok_logger) as cl:
        output = tokenizer(examples[text_column_name])
    # clm input could be much much longer than block_size
    if "Token indices sequence length is longer than the" in cl.out:
        tok_logger.warning(
            "^^^^^^^^^^^^^^^^ Please ignore the warning above - this long input will be chunked into smaller bits"
            " before being passed to the model."
        )
    return output

with training_args.main_process_first(desc="dataset map tokenization"):
    if not data_args.streaming:
        tokenized_datasets = raw_datasets.map(
            tokenize_function,
            batched=True,
            num_proc=data_args.preprocessing_num_workers,
            remove_columns=column_names,
            load_from_cache_file=not data_args.overwrite_cache,
            desc="Running tokenizer on dataset",
        )
    else:
        tokenized_datasets = raw_datasets.map(
            tokenize_function,
            batched=True,
            remove_columns=column_names,
        )

if data_args.block_size is None:
    block_size = tokenizer.model_max_length
    if block_size > 1024:
        logger.warning(
            "The chosen tokenizer supports a `model_max_length` that is longer than the default `block_size` value"
            " of 1024. If you would like to use a longer `block_size` up to `tokenizer.model_max_length` you can"
            " override this default with `--block_size xxx`."
        )
        block_size = 1024
else:
    if data_args.block_size > tokenizer.model_max_length:
        logger.warning(
            f"The block_size passed ({data_args.block_size}) is larger than the maximum length for the model"
            f"({tokenizer.model_max_length}). Using block_size={tokenizer.model_max_length}."
        )
    block_size = min(data_args.block_size, tokenizer.model_max_length)


06/09/2023 17:25:12 - INFO - datasets.arrow_dataset - Process #0 will write at /Users/joon/.cache/huggingface/datasets/machelreid___m2d2/Religion_and_belief_systems/0.0.0/eb235f33a5de3163c10549b7f63c906910539c8a8c0ec5ade1285ccbf5067d00/cache-57a76217e55c2e9f_00000_of_00008.arrow
06/09/2023 17:25:12 - INFO - datasets.arrow_dataset - Process #1 will write at /Users/joon/.cache/huggingface/datasets/machelreid___m2d2/Religion_and_belief_systems/0.0.0/eb235f33a5de3163c10549b7f63c906910539c8a8c0ec5ade1285ccbf5067d00/cache-57a76217e55c2e9f_00001_of_00008.arrow
06/09/2023 17:25:12 - INFO - datasets.arrow_dataset - Process #2 will write at /Users/joon/.cache/huggingface/datasets/machelreid___m2d2/Religion_and_belief_systems/0.0.0/eb235f33a5de3163c10549b7f63c906910539c8a8c0ec5ade1285ccbf5067d00/cache-57a76217e55c2e9f_00002_of_00008.arrow
06/09/2023 17:25:12 - INFO - datasets.arrow_dataset - Process #3 will write at /Users/joon/.cache/huggingface/datasets/machelreid___m2d2/Religion_and_belief_sys

## group_texts

In [15]:
# Main data processing function that will concatenate all texts from our dataset and generate chunks of block_size.
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

# Note that with `batched=True`, this map processes 1,000 texts together, so group_texts throws away a remainder
# for each of those groups of 1,000 texts. You can adjust that batch_size here but a higher value might be slower
# to preprocess.
#
# To speed up this part, we use multiprocessing. See the documentation of the map method for more information:
# https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.map

with training_args.main_process_first(desc="grouping texts together"):
    if not data_args.streaming:
        lm_datasets = tokenized_datasets.map(
            group_texts,
            batched=True,
            num_proc=data_args.preprocessing_num_workers,
            load_from_cache_file=not data_args.overwrite_cache,
            desc=f"Grouping texts in chunks of {block_size}",
        )
    else:
        lm_datasets = tokenized_datasets.map(
            group_texts,
            batched=True,
        )



06/09/2023 17:25:12 - INFO - datasets.arrow_dataset - Process #0 will write at /Users/joon/.cache/huggingface/datasets/machelreid___m2d2/Religion_and_belief_systems/0.0.0/eb235f33a5de3163c10549b7f63c906910539c8a8c0ec5ade1285ccbf5067d00/cache-f210143e3f68275d_00000_of_00008.arrow
06/09/2023 17:25:12 - INFO - datasets.arrow_dataset - Process #1 will write at /Users/joon/.cache/huggingface/datasets/machelreid___m2d2/Religion_and_belief_systems/0.0.0/eb235f33a5de3163c10549b7f63c906910539c8a8c0ec5ade1285ccbf5067d00/cache-f210143e3f68275d_00001_of_00008.arrow
06/09/2023 17:25:12 - INFO - datasets.arrow_dataset - Process #2 will write at /Users/joon/.cache/huggingface/datasets/machelreid___m2d2/Religion_and_belief_systems/0.0.0/eb235f33a5de3163c10549b7f63c906910539c8a8c0ec5ade1285ccbf5067d00/cache-f210143e3f68275d_00002_of_00008.arrow
06/09/2023 17:25:12 - INFO - datasets.arrow_dataset - Process #3 will write at /Users/joon/.cache/huggingface/datasets/machelreid___m2d2/Religion_and_belief_sys

# RESULTS

## Raw dataset

In [16]:
raw_datasets["validation"]["text"][:5]

['',
 'William Henry Willimon',
 '',
 'William Henry Willimon (born May 15, 1946) is an American theologian and bishop in the United Methodist Church, retired, who for eight years served the North Alabama Conference. He is currently Professor of the Practice of Christian Ministry and Director of the Doctor of Ministry program at Duke Divinity School. He is former Dean of the Chapel at Duke University and is considered by many as one of America\'s best-known and most influential preachers. A Pulpit & Pew Research on Pastoral Leadership survey determined that he was one of the two most frequently read writers by pastors in mainline Protestantism alongside the Roman Catholic writer Henri Nouwen. His books have sold over a million copies. He is also Editor-At-Large of "The Christian Century". His 2019 memoir "Accidental Preacher" was released to wide acclaim, described by Justo L. Gonzalez as "An exceptional example of theology at its best."',
 'Biography.']

## Tokenisation

In [17]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 1378270
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 108776
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 108776
    })
})

In [18]:
print(tokenized_datasets["validation"]["input_ids"][:5])

[[], [17121, 8616, 2561, 20473], [], [17121, 8616, 2561, 20473, 357, 6286, 1737, 1315, 11, 22717, 8, 318, 281, 1605, 37275, 666, 290, 24233, 287, 262, 1578, 38029, 4564, 11, 9880, 11, 508, 329, 3624, 812, 4983, 262, 2258, 9266, 8785, 13, 679, 318, 3058, 8129, 286, 262, 19939, 286, 4302, 9475, 290, 5890, 286, 262, 9356, 286, 9475, 1430, 379, 11083, 33170, 3961, 13, 679, 318, 1966, 11325, 286, 262, 32939, 379, 11083, 2059, 290, 318, 3177, 416, 867, 355, 530, 286, 2253, 338, 1266, 12, 4002, 290, 749, 14212, 662, 17892, 13, 317, 21624, 15544, 1222, 21805, 4992, 319, 11303, 6864, 26935, 5526, 5295, 326, 339, 373, 530, 286, 262, 734, 749, 6777, 1100, 8786, 416, 43978, 287, 50114, 25310, 1042, 7848, 262, 7993, 7835, 6260, 44485, 44599, 21006, 13, 2399, 3835, 423, 2702, 625, 257, 1510, 9088, 13, 679, 318, 635, 12058, 12, 2953, 12, 21968, 286, 366, 464, 4302, 13641, 1911, 2399, 13130, 24649, 366, 17320, 35182, 3771, 3493, 1, 373, 2716, 284, 3094, 21684, 11, 3417, 416, 2329, 78, 406, 13, 24416, 

In [19]:
print(tokenizer.convert_ids_to_tokens(chain(*tokenized_datasets["validation"]["input_ids"][:5])))

['William', 'ĠHenry', 'ĠWill', 'imon', 'William', 'ĠHenry', 'ĠWill', 'imon', 'Ġ(', 'born', 'ĠMay', 'Ġ15', ',', 'Ġ1946', ')', 'Ġis', 'Ġan', 'ĠAmerican', 'Ġtheolog', 'ian', 'Ġand', 'Ġbishop', 'Ġin', 'Ġthe', 'ĠUnited', 'ĠMethodist', 'ĠChurch', ',', 'Ġretired', ',', 'Ġwho', 'Ġfor', 'Ġeight', 'Ġyears', 'Ġserved', 'Ġthe', 'ĠNorth', 'ĠAlabama', 'ĠConference', '.', 'ĠHe', 'Ġis', 'Ġcurrently', 'ĠProfessor', 'Ġof', 'Ġthe', 'ĠPractice', 'Ġof', 'ĠChristian', 'ĠMinistry', 'Ġand', 'ĠDirector', 'Ġof', 'Ġthe', 'ĠDoctor', 'Ġof', 'ĠMinistry', 'Ġprogram', 'Ġat', 'ĠDuke', 'ĠDivinity', 'ĠSchool', '.', 'ĠHe', 'Ġis', 'Ġformer', 'ĠDean', 'Ġof', 'Ġthe', 'ĠChapel', 'Ġat', 'ĠDuke', 'ĠUniversity', 'Ġand', 'Ġis', 'Ġconsidered', 'Ġby', 'Ġmany', 'Ġas', 'Ġone', 'Ġof', 'ĠAmerica', "'s", 'Ġbest', '-', 'known', 'Ġand', 'Ġmost', 'Ġinfluential', 'Ġpre', 'achers', '.', 'ĠA', 'ĠPul', 'pit', 'Ġ&', 'ĠPew', 'ĠResearch', 'Ġon', 'ĠPast', 'oral', 'ĠLeadership', 'Ġsurvey', 'Ġdetermined', 'Ġthat', 'Ġhe', 'Ġwas', 'Ġone', 'Ġof', 'Ġth

In [20]:
tokenizer.convert_ids_to_tokens([198])

['Ċ']

In [21]:
len(list(chain(*tokenized_datasets["validation"]["input_ids"])))

5414308

## Grouping Text

In [22]:
lm_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 86291
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 5230
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 5256
    })
})

In [23]:
lm_datasets['validation'][0]['input_ids'][:5]

[17121, 8616, 2561, 20473, 17121]

In [24]:
print(tokenizer.convert_ids_to_tokens(lm_datasets['validation'][0]['input_ids'][:5]))


['William', 'ĠHenry', 'ĠWill', 'imon', 'William']


In [25]:
len(lm_datasets['validation'])

5230

# Training & Evaluation

## set Trainer

In [26]:
# Trainer 설정
#   0) compute_metrics 설정 가능 (위에서 보셈)
#   1) callbacks에 대한 설정은 없네? # TODO

if training_args.do_train:
    if "train" not in tokenized_datasets:
        raise ValueError("--do_train requires a train dataset")
    train_dataset = lm_datasets["train"]
    if data_args.max_train_samples is not None:
        max_train_samples = min(len(train_dataset), data_args.max_train_samples)
        train_dataset = train_dataset.select(range(max_train_samples))

if training_args.do_eval:
    if "validation" not in tokenized_datasets:
        raise ValueError("--do_eval requires a validation dataset")
    eval_dataset = lm_datasets["validation"]
    if data_args.max_eval_samples is not None:
        max_eval_samples = min(len(eval_dataset), data_args.max_eval_samples)
        eval_dataset = eval_dataset.select(range(max_eval_samples))

    def preprocess_logits_for_metrics(logits, labels):
        if isinstance(logits, tuple):
            # Depending on the model and config, logits may contain extra tensors,
            # like past_key_values, but logits always come first
            logits = logits[0]
        return logits.argmax(dim=-1)
    from datetime import datetime

    metric = evaluate.load("accuracy", experiment_id=datetime.now().strftime(format='%y%m%d_%H%M') )  # TODO

    def compute_metrics(eval_preds):
        preds, labels = eval_preds
        # preds have the same shape as the labels, after the argmax(-1) has been calculated
        # by preprocess_logits_for_metrics but we need to shift the labels
        labels = labels[:, 1:].reshape(-1)
        preds = preds[:, :-1].reshape(-1)
        return metric.compute(predictions=preds, references=labels)

# Initialize our Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset if training_args.do_train else None,
    eval_dataset=eval_dataset if training_args.do_eval else None,
    tokenizer=tokenizer,
    # Data collator will default to DataCollatorWithPadding, so we change it.
    data_collator=default_data_collator,
    compute_metrics=compute_metrics if training_args.do_eval and not is_torch_tpu_available() else None,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics
    if training_args.do_eval and not is_torch_tpu_available()
    else None,
    callbacks=[EarlyStoppingCallback(2)],   # TODO
)

## training

In [27]:
# Training: resume_from_checkpoint or last_checkpoint 있으면 불러와서 거기서 부터 시작
# Training:
if training_args.do_train:
    checkpoint = None
    if training_args.resume_from_checkpoint is not None:
        checkpoint = training_args.resume_from_checkpoint
    elif last_checkpoint is not None:
        checkpoint = last_checkpoint
    train_result = trainer.train(resume_from_checkpoint=checkpoint)
    trainer.save_model()  # Saves the tokenizer too for easy upload

    metrics = train_result.metrics

    max_train_samples = (
        data_args.max_train_samples if data_args.max_train_samples is not None else len(train_dataset)
    )
    metrics["train_samples"] = min(max_train_samples, len(train_dataset))

    trainer.log_metrics("train", metrics)
    trainer.save_metrics("train", metrics)
    trainer.save_state()


## evaluation

In [28]:

# Evaluation: resume_from_checkpoint or last_checkpoint 있으면 불러와서 거기서 부터 시작
# Evaluation
if training_args.do_eval:
    logger.info("*** Evaluate ***")

    metrics = trainer.evaluate()

    max_eval_samples = data_args.max_eval_samples if data_args.max_eval_samples is not None else len(eval_dataset)
    metrics["eval_samples"] = min(max_eval_samples, len(eval_dataset))
    try:
        perplexity = math.exp(metrics["eval_loss"])
    except OverflowError:
        perplexity = float("inf")
    metrics["perplexity"] = perplexity

    trainer.log_metrics("eval", metrics)
    trainer.save_metrics("eval", metrics)

kwargs = {"finetuned_from": model_args.model_name_or_path, "tasks": "text-generation"}
if data_args.dataset_name is not None:
    kwargs["dataset_tags"] = data_args.dataset_name
    if data_args.dataset_config_name is not None:
        kwargs["dataset_args"] = data_args.dataset_config_name
        kwargs["dataset"] = f"{data_args.dataset_name} {data_args.dataset_config_name}"
    else:
        kwargs["dataset"] = data_args.dataset_name

# if training_args.push_to_hub:
#     trainer.push_to_hub(**kwargs)
# else:
#     trainer.create_model_card(**kwargs)


# def _mp_fn(index):
# # For xla_spawn (TPUs)
# main()


# if __name__ == "__main__":
# main()


06/09/2023 17:25:19 - INFO - __main__ - *** Evaluate ***


[INFO|trainer.py:3129] 2023-06-09 17:25:19,681 >> ***** Running Evaluation *****
[INFO|trainer.py:3131] 2023-06-09 17:25:19,682 >>   Num examples = 5230
[INFO|trainer.py:3134] 2023-06-09 17:25:19,682 >>   Batch size = 8


ValueError: Predictions and/or references don't match the expected format.
Expected format: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)},
Input predictions: [-9223372036854775808 -9223372036854775808 -9223372036854775808 ...
 -9223372036854775808 -9223372036854775808 -9223372036854775808],
Input references: [ 8616  2561 20473 ... 11307    13  4863]

In [ ]:
metrics